## 8. Classes and Objects

有很多时候，这些模式并不是必须的。

比如代理模式。 如果一定要用代理模式，就需要有一个用代理模式充分的理由。如果要用抽象类，就必须有一个用抽象类的理由。 在敏捷开发中，先实现功能，然后再不断的优化。

软件工程是为了解决问题。

### 8.1 改变实例的 string representation

其实就是修改 `__str__` 和 `__repr__` 方法。

一个用于 str表示，另一个用于对象表示

In [3]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '({0.x}, {0.y})'.format(self)

p = Pair(3, 4)
print(str(p))
print(repr(p))
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))

(3, 4)
Pair(3, 4)
p is Pair(3, 4)
p is (3, 4)


### 8.2 自定义实例的 format

其实就是 `__format__` 方法，用于 format

In [5]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

d = Date(2018, 4, 1)
print(format(d, 'ymd'))
print(format(d, 'mdy'))
print(format(d, 'dmy'))

2018-4-1
4/1/2018
1/4/2018


### 8.3 Context-Management Protocol

这个比较重要，其实就是定义好 进入、退出方法： `__enter__` 和 `__exit__`

Context的作用就是定义一种上下文，提供一种环境，例如 对某个文件的操作； 例如建立某个连接（数据库连接之类）； 例如tensorflow的session。

其实大部分时候，在干一件事情之前，需要准备一些配置； 在干完事情之后，清理一下配置。 这就是建立环境（context）的作用。 为了避免代码重复。

可以用 with statement

In [6]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

if __name__ == '__main__':
    from functools import partial

    c = LazyConnection(('www.python.org', 80))
    # Connection closed
    with c as s:
        # c.__enter__() executes: connection open
        s.send(b'GET /index.html HTTP/1.0\r\n')
        s.send(b'Host: www.python.org\r\n')
        s.send(b'\r\n')
        resp = b''.join(iter(partial(s.recv, 8192), b''))
        # c.__exit__() executes: connection closed

    print('Got %d bytes' % len(resp))


Got 391 bytes


In [8]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.connections = []

    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock
                
    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

if __name__ == '__main__':
    # Example use
    from functools import partial

    conn = LazyConnection(('www.python.org', 80))
    with conn as s:
        s.send(b'GET /index.html HTTP/1.0\r\n')
        s.send(b'Host: www.python.org\r\n')
        s.send(b'\r\n')
        resp = b''.join(iter(partial(s.recv, 8192), b''))

    print('Got %d bytes' % len(resp))

    # 同时存在两个连接
    with conn as s1, conn as s2:
        s1.send(b'GET /downloads HTTP/1.0\r\n')
        s2.send(b'GET /index.html HTTP/1.0\r\n')
        s1.send(b'Host: www.python.org\r\n')
        s2.send(b'Host: www.python.org\r\n')
        s1.send(b'\r\n')
        s2.send(b'\r\n')
        resp1 = b''.join(iter(partial(s1.recv, 8192), b''))
        resp2 = b''.join(iter(partial(s2.recv, 8192), b''))

    print('resp1 got %d bytes' % len(resp1))
    print('resp2 got %d bytes' % len(resp2))


Got 391 bytes
resp1 got 390 bytes
resp2 got 391 bytes


### 8.4 如果需要创建大量实例，一种节省内存的办法

定义`__slots__`， 就没有 `__dict__` 占用空间了。

但是还是有点奇怪。



In [17]:
class Date0(object):
    pass

class Date1(object):
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
class Date2(object):
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

import sys

print(sys.getsizeof(Date0()))
d1 = Date1(2000, 1, 1)
print(d1, sys.getsizeof(d1))
d2 = Date2(2000, 1, 1)
print(d2, sys.getsizeof(d2))

56
<__main__.Date1 object at 0x108b7c978> 56
<__main__.Date2 object at 0x108b81288> 64


In [18]:
d1.__dict__

{'day': 1, 'month': 1, 'year': 2000}

In [26]:
dir(d2)
print(id(d2.__slots__))
print(id(Date2(1, 2, 3).__slots__))

4441200712
4441200712


### 8.5  实例的'private'方法

实际上，python不是编译语言。 所有代码可见，想怎么调用怎么调用。。。

一种方法是加上前缀 `_` 或者两个 `__`

### 8.6 一种类似于 C#、Java 定义 property的方法

避免直接操作 'private' 属性。一种保护机制。

In [31]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    # Getter function
    @property
    def first_name(self):
        return self._first_name

    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

if __name__ == '__main__':
    a = Person('Guido')
    print(a.first_name)
    a.first_name = 'Dave'
    print(a.first_name)
    try:
        a.first_name = 42
    except TypeError as e:
        print(e)


Guido
Dave
Expected a string


### 8.7 调用父类方法

调用 `super()` 可以获取父类方法。

常做的是调用父类的 `__init__` ，用于初始化

In [32]:
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam()      # Call parent spam()

if __name__ == '__main__':
    b = B()
    b.spam()


B.spam
A.spam


In [33]:
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

if __name__ == '__main__':
    b = B()
    print(b.x, b.y)


0 1


下面这个例子，是代理模式，就是不直接调用对象，使用代理去访问对象。

In [35]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    # Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        return getattr(self._obj, name)

    # Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)    # Call original __setattr__
        else:
            setattr(self._obj, name, value)

if __name__ == '__main__':
    class A:
        def __init__(self, x):
            self.x = x
        def spam(self):
            print('A.spam')

    a = A(42)
    p = Proxy(a)
    print(p.x)
    print(p.spam())
    p.x = 37
    print('Should be 37:', p.x)
    print('Should be 37:', a.x)


42
A.spam
None
Should be 37: 37
Should be 37: 37


In [36]:

class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')

class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C.__init__')

if __name__ == '__main__':
    # Please observe double call of Base.__init__
    # Base.__init__ 被调用了两次， A、B各一次
    c = C()


Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


In [39]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')

class C(A,B):
    def __init__(self):
        super().__init__()     # Only one call to super() here
        print('C.__init__')

if __name__ == '__main__':
    # Observe that each class initialized only once
    # 这里只被调用了一次。。不明所以
    c = C()
    print(C.__mro__)


Base.__init__
B.__init__
A.__init__
C.__init__
(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>)



注意这里 MRO（method resolution order），是 C->A->B->Base， 所以在用 `__init__` 方法时，也是按照这个顺序。

可以参考博文： <https://rhettinger.wordpress.com/2011/05/26/super-considered-super/> 。不过有点老。。

### 8.8 在子类中扩展 property的功能

其实就是继承，重写，类似 override

In [40]:
# Example of managed attributes via properties

class Person:
    def __init__(self, name):
        self.name = name

    # Getter function
    @property
    def name(self):
        return self._name

    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value

    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name

    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

class SubPerson2(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson2, SubPerson2).name.__set__(self, value)

class SubPerson3(Person):
    #@property
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name

if __name__ == '__main__':
    a = Person('Guido')
    print(a.name)
    a.name = 'Dave'
    print(a.name)
    try:
        a.name = 42
    except TypeError as e:
        print(e)


Guido
Dave
Expected a string


### 8.9 还是封装，一种检查赋值类型的操作

下面这个例子的思想就是， 为了避免赋值类型错误而进行的一个检查， 把这种方法封装起来。

第二个例子更加通用。。并且用到了装饰器。 在装饰器方法里，直接用 `setattr(cls, name, Typed(name, expected_type))` 设置属性

In [41]:
# Descriptor attribute for an integer type-checked attribute
class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]

class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y

if __name__ == '__main__':
    p = Point(2, 3)
    print(p.x)
    p.y = 5
    try:
        p.x = 2.3
    except TypeError as e:
        print(e)


2
Expected an int


In [ ]:
# Descriptor for a type-checked attribute
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected ' + str(self.expected_type))
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]

# Class decorator that applies it to selected attributes
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            # Attach a Typed descriptor to the class
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate

# Example use
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

if __name__ == '__main__':
    s = Stock('ACME', 100, 490.1)
    print(s.name, s.shares, s.price)
    s.shares = 50
    try:
        s.shares = 'a lot'
    except TypeError as e:
        print(e)


### 8.10  Lazy模式

就是初始化的时候不计算。用到的时候在计算，然后缓存。

In [58]:
class lazyproperty:
    def __init__(self, func):
        print(self)
        print(func.__name__)
        self.func = func
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            print('hello', self, instance)
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

import math
class Circle:
    def __init__(self, radius):
        self.radius = radius

    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius
        
if __name__ == '__main__':
    c = Circle(4.0)
    print(c.radius)
    # 第一次计算，因为没有，确实调用了方法
    print(c.__getattribute__('area'))
#     print(c.area)
    
#     print(c.perimeter)
#     # 第二次计算，就直接用缓存了
#     print(c.area)
#     print(c.perimeter)

area
perimeter
4.0
hello <__main__.lazyproperty object at 0x108c62dd8> <__main__.Circle object at 0x108c62f28>
Computing area
50.26548245743669


理解一下上面的过程：

1. 开始的时候， Circle实例有两个属性 area和 perimeter，都是 lazyproperty 实例。
2. 第一次调用的时候，执行了实际的方法；结束后，直接将Circle实例的属性给替换成实际的值了，不再是 lazyproperty实例。 lazyproperty 自己把自己给干掉了。。。。


In [60]:
def lazyproperty(func):
    name = '_lazy_' + func.__name__
    @property
    def lazy(self):
        if hasattr(self, name):
            return getattr(self, name)
        else:
            value = func(self)
            setattr(self, name, value)
            return value
    return lazy
     
import math
class Circle:
    def __init__(self, radius):
        self.radius = radius

    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius
    
if __name__ == '__main__':
    c = Circle(4.0)
    print(c.radius)
    # 第一次计算，因为没有，确实调用了方法
    print(c.area)
    
    print(c.perimeter)
    # 第二次计算，就直接用缓存了
    print(c.area)
    print(c.perimeter)

4.0
Computing area
50.26548245743669
Computing perimeter
25.132741228718345
50.26548245743669
25.132741228718345


上面这个例子，自己没把自己干掉，不过是换了个名字，代理了一下。


### 8.11 简化 __init__ 方法

这里用到了 `setattr` ，不过个人并不喜欢这样做。这样固然方便，但是隐藏很多细节，容易出错。

遇到一大批类需要这样定义的时候，可以用这种技巧简化代码。。

In [61]:
class Structure:
    # Class variable that specifies expected fields
    _fields= []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))

        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
        
# Example class definitions
if __name__ == '__main__':
    class Stock(Structure):
        _fields = ['name', 'shares', 'price']

    class Point(Structure):
        _fields = ['x','y']

    class Circle(Structure):
        _fields = ['radius']
        def area(self):
            return math.pi * self.radius ** 2

if __name__ == '__main__':
    s = Stock('ACME', 50, 91.1)
    print(s.name, s.shares, s.price)

    p = Point(2,3)
    print(p.x, p.y)

    c = Circle(4.5)
    print(c.radius)

    try:
        s2 = Stock('ACME', 50)
    except TypeError as e:
        print(e)


ACME 50 91.1
2 3
4.5
Expected 3 arguments



### 8.12 抽象类的使用 （Abstract Base Class），定义接口

在抽象类的方法里面可以使用  `NotImplementError` 这种异常。

或者直接用 `from abc import ABCMeta, abstractmethod`这种方式。


在一种大规模软件工程的模式下，确实需要定义接口，以便于交互。

In [63]:
# Defining a simple abstract base class

from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    @abstractmethod
    def write(self, data):
        pass

# Example implementation
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        print('reading')
    def write(self, data):
        print('writing')

# Example of type checking
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream')
    print('serializing')

# Examples
if __name__ == '__main__':
    # Attempt to instantiate ABC directly (doesn't work)
    try:
        a = IStream()
    except TypeError as e:
        print(e)

    # Instantiation of a concrete implementation
    a = SocketStream()
    a.read()
    a.write('data')

    # Passing to type-check function
    serialize(None, a)

    # Attempt to pass a file-like object to serialize (fails)
    import sys

    try:
        serialize(None, sys.stdout)
    except TypeError as e:
        print(e)

    # Register file streams and retry
    import io
    IStream.register(io.IOBase)

    serialize(None, sys.stdout)

Can't instantiate abstract class IStream with abstract methods read, write
reading
writing
serializing
Expected an IStream
serializing


In [64]:
from abc import ABCMeta, abstractmethod

class A(metaclass=ABCMeta):
    @property
    @abstractmethod
    def name(self):
        pass

    @name.setter
    @abstractmethod
    def name(self, value):
        pass

    @classmethod
    @abstractmethod
    def method1(cls):
        pass

    @staticmethod
    @abstractmethod
    def method2():
        pass



### 8.13 Data Model 或者 Type System，用于类型约束

下面例子用了三种方法

1. 直接声明
2. 装饰器
3. metaclass

In [65]:
# Base class. Uses a descriptor to set a value
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        self.__dict__.update(opts)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

# Descriptor for enforcing types
class Typed(Descriptor):
    expected_type = type(None)

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, value)

# Descriptor for enforcing values
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)

class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        self.size = opts['size']
        super().__init__(name, **opts)

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)

class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

class SizedString(String, MaxSized):
    pass

# Class decorator to apply constraints
def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
        return cls
    return decorate

# A metaclass that applies checking
class checkedmeta(type):
    def __new__(cls, clsname, bases, methods):
        # Attach attribute names to the descriptors
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                value.name = key
        return type.__new__(cls, clsname, bases, methods)

# Testing code
def test(s):
    print(s.name)
    s.shares = 75
    print(s.shares)
    try:
        s.shares = -10
    except ValueError as e:
        print(e)
    try:
        s.price = 'a lot'
    except TypeError as e:
        print(e)

    try:
        s.name = 'ABRACADABRA'
    except ValueError as e:
        print(e)

# Various Examples:
if __name__ == '__main__':
    print("# --- Class with descriptors")
    class Stock:
        # Specify constraints
        name = SizedString('name',size=8)
        shares = UnsignedInteger('shares')
        price = UnsignedFloat('price')
        def __init__(self, name, shares, price):
            self.name = name
            self.shares = shares
            self.price = price

    s = Stock('ACME',50,91.1)
    test(s)

    print("# --- Class with class decorator")
    @check_attributes(name=SizedString(size=8), 
                      shares=UnsignedInteger,
                      price=UnsignedFloat)
    class Stock:
        def __init__(self, name, shares, price):
            self.name = name
            self.shares = shares
            self.price = price

    s = Stock('ACME',50,91.1)
    test(s)

    print("# --- Class with metaclass")
    class Stock(metaclass=checkedmeta):
        name   = SizedString(size=8)
        shares = UnsignedInteger()
        price  = UnsignedFloat()
        def __init__(self, name, shares, price):
            self.name = name
            self.shares = shares
            self.price = price

    s = Stock('ACME',50,91.1)
    test(s)
        


# --- Class with descriptors
ACME
75
Expected >= 0
expected <class 'float'>
size must be < 8
# --- Class with class decorator
ACME
75
Expected >= 0
expected <class 'float'>
size must be < 8
# --- Class with metaclass
ACME
75
Expected >= 0
expected <class 'float'>
size must be < 8


### 8.14 自定义容器（container）

包括：

1. 继承自 `Iterable`
2. `__getitem__`， 序列容器，可以索引
3. `__len__`，长度
4. `__setitem__`， 序列容器，可以索引修改


下面例子中的 bisect，可以用来维持顺序。

In [66]:
import collections
import bisect

class SortedItems(collections.Sequence):
    def __init__(self, initial=None):
        self._items = sorted(initial) if initial is not None else []

    # Required sequence methods
    def __getitem__(self, index):
        return self._items[index]

    def __len__(self):
        return len(self._items)

    # Method for adding an item in the right location
    def add(self, item):
        bisect.insort(self._items, item)

if __name__ == '__main__':
    items = SortedItems([5, 1, 3])
    print(list(items))
    print(items[0])
    print(items[-1])
    items.add(2)
    print(list(items))
    items.add(-10)
    print(list(items))
    print(items[1:4])
    print(3 in items)
    print(len(items))
    for n in items:
        print(n)


[1, 3, 5]
1
5
[1, 2, 3, 5]
[-10, 1, 2, 3, 5]
[1, 2, 3]
True
5
-10
1
2
3
5


In [67]:
import collections

class Items(collections.MutableSequence):
    def __init__(self, initial=None):
        self._items = list(initial) if initial is not None else []

    # Required sequence methods
    def __getitem__(self, index):
        print('Getting:', index)
        return self._items[index]

    def __setitem__(self, index, value):
        print('Setting:', index, value)
        self._items[index] = value

    def __delitem__(self, index):
        print('Deleting:', index)
        del self._items[index]

    def insert(self, index, value):
        print('Inserting:', index, value)
        self._items.insert(index, value)

    def __len__(self):
        print('Len')
        return len(self._items)

if __name__ == '__main__':
    a = Items([1, 2, 3])
    print(len(a))
    a.append(4)
    a.append(2)
    print(a.count(2))
    a.remove(3)


Len
3
Len
Inserting: 3 4
Len
Inserting: 4 2
Getting: 0
Getting: 1
Getting: 2
Getting: 3
Getting: 4
Getting: 5
2
Getting: 0
Getting: 1
Getting: 2
Deleting: 2


### 8.15 属性委托访问（delegating attribute access）

也是属于proxy模式的一种。不直接访问。 间接访问

### 8.16 创建实例

- 除了 `__init__`方法外，还有 `classmethod`
- 还可以用 `__new__`。 理论上，  `__new__` 比 `__init__` 更底层，除了构建一个对象，还没有设置任何属性

### 8.17 Mixin

Mixin 类似于插件， 而且在一定程度上可以“通用”的插件。 它扩展一些功能，而不改变原类的设计。

可以这样做： 设计一些Mixin，适用于某个类系列。

In [68]:
class LoggedMappingMixin:
    '''
    Add logging to get/set/delete operations for debugging.
    '''
    __slots__ = ()

    def __getitem__(self, key):
        print('Getting ' + str(key))
        return super().__getitem__(key)

    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return super().__setitem__(key, value)

    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)
    
print('# ---- LoggedDict Example')

class LoggedDict(LoggedMappingMixin, dict):
    pass

d = LoggedDict()
d['x'] = 23
print(d['x'])
del d['x']

# ---- LoggedDict Example
Setting x = 23
Getting x
23
Deleting x


In [69]:
class SetOnceMappingMixin:
    '''
    Only allow a key to be set once.
    '''
    __slots__ = ()
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key) + ' already set')
        return super().__setitem__(key, value)

print('# ---- SetOnceDefaultDict Example')

from collections import defaultdict
class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict):
    pass
 
d = SetOnceDefaultDict(list)
d['x'].append(2)
d['y'].append(3)
d['x'].append(10)
try:
    d['x'] = 23
except KeyError as e:
    print(e)


# ---- SetOnceDefaultDict Example
'x already set'


In [70]:
class StringKeysMappingMixin:
    '''
    Restrict keys to strings only
    '''
    __slots__ = ()
    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        return super().__setitem__(key, value)

print('# ---- StringOrderedDict Example')
from collections import OrderedDict

class StringOrderedDict(StringKeysMappingMixin,
                        SetOnceMappingMixin,
                        OrderedDict):
    pass

d = StringOrderedDict()
d['x'] = 23
try:
    d[42] = 10
except TypeError as e:
    print(e)

try:
    d['x'] = 42
except KeyError as e:
    print(e)


# ---- StringOrderedDict Example
keys must be strings
'x already set'


In [71]:
class RestrictKeysMixin:
    def __init__(self, *args, _restrict_key_type, **kwargs):
        self.__restrict_key_type = _restrict_key_type
        super().__init__(*args, **kwargs)
    
    def __setitem__(self, key, value):
        if not isinstance(key, self.__restrict_key_type):
            raise TypeError('Keys must be ' + str(self.__restrict_key_type))
        super().__setitem__(key, value)

# Example

class RDict(RestrictKeysMixin, dict):
    pass
 
d = RDict(_restrict_key_type=str)
e = RDict([('name','Dave'), ('n',37)], _restrict_key_type=str)
f = RDict(name='Dave', n=37, _restrict_key_type=str)
print(f)
try:
    f[42] = 10
except TypeError as e:
    print(e)


{'name': 'Dave', 'n': 37}
Keys must be <class 'str'>



### 8.19 状态机

状态机是这样子的： 有多种状态， 每种状态响应特定动作，进行状态转移。

一种实现方式是： 记录状态，每次转移的时候进行条件判断。 这是过程式的编写方式。 一个好处是，直接，但是对于复杂的状态机模型，可以能回很难看，很难维护。另一个好处是在条件判断处可以设置锁，实现并发。

另一种思路是： 每个状态一个类，对每个动作响应，状态迁移实现一个方法。这样做的一个好处是直观，并且便于维护。 但是缺点是代码量较大。 对于并发，需要额外的代码进行处理。

In [72]:
class Connection:
    def __init__(self):
        self.new_state(ClosedConnection)

    def new_state(self, state):
        self.__class__ = state

    def read(self):
        raise NotImplementedError()

    def write(self, data):
        raise NotImplementedError()

    def open(self):
        raise NotImplementedError()

    def close(self):
        raise NotImplementedError()

class ClosedConnection(Connection):
    def read(self):
        raise RuntimeError('Not open')

    def write(self, data):
        raise RuntimeError('Not open')

    def open(self):
        self.new_state(OpenConnection)

    def close(self):
        raise RuntimeError('Already closed')

class OpenConnection(Connection):
    def read(self):
        print('reading')

    def write(self, data):
        print('writing')

    def open(self):
        raise RuntimeError('Already open')

    def close(self):
        self.new_state(ClosedConnection)

# Example
if __name__ == '__main__':
    c = Connection()
    print(c)
    try:
        c.read()
    except RuntimeError as e:
        print(e)

    c.open()
    print(c)
    c.read()
    c.close()
    print(c)


Not open
reading



### 8.20 通过名称字符串调用方法

其实方法啊、函数啊，都是一个对象，在类里面就是一个属性。 用 `getattr`就可以了

### 8.23 Cyclic Data Structure 内存管理


这个问题涉及 循环引用 的内存管理。 比如一些树、图的数据结构，就有循环引用。

In [104]:
class Data:
    def __del__(self):
        print('Data.__del__')

class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []
    def add_child(self, child):
        self.children.append(child)
        child.parent = self

In [105]:
# 没有循环引用时正常
a = Data()
del a
a = Node()
del a

Data.__del__
Data.__del__


In [107]:
# 这里删除了父节点，子节点仍然保留对父节点的引用，所以不会释放
a = Node()
a.add_child(Node())
del a
# 但是子节点也没有引用，这里就留下了 2个垃圾

In [110]:
import gc
gc.collect()   # 必须强制垃圾回收，才能删除

1075

In [111]:
a = Node()
a.add_child(Node())
a_ref = weakref.ref(a)
a_ref   

<weakref at 0x108c682c8; to 'Node' at 0x108b46470>

In [112]:
a_ref()

a_ref 实际上就是 Node的一个指针，并且没有引用计数。 当删掉Node 之后，这个指针实际上指向一个空地址，即返回None。（上面的例子中，由于子节点保留了对父节点的引用，所以删除父节点时，weakref仍有效，因为没有真正删掉）

所以为了避免循环引用，在反向上（比如指向父节点），可以使用  weakref。 这样在删除节点时，就能彻底删除。避免内存泄漏。（见下面的例子）

In [116]:
import weakref

class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return 'Node({!r:})'.format(self.value)

    # property that manages the parent as a weak-reference
    @property
    def parent(self):
        return self._parent if self._parent is None else self._parent()

    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

if __name__ == '__main__':
    root = Node('parent')
    c1 = Node('c1')
    c2 = Node('c2')
    root.add_child(c1)
    root.add_child(c2)

    print(c1.parent)
    del root
    print(c1.parent)
    print(c2.parent)
    gc.collect()

Node('parent')
None
None


### 8.24 自定义比较操作

- `__le__`
- `__ge__`
- `__gt__`
- `__lt__`
- `__eq__`
等等

In [117]:
from functools import total_ordering
class Room:
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width

@total_ordering
class House:
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()

    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)

    def add_room(self, room):
        self.rooms.append(room)

    def __str__(self):
        return '{}: {} square foot {}'.format(self.name, 
                                              self.living_space_footage, 
                                              self.style)

    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage

    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage 



# Build a few houses, and add rooms to them.
h1 = House('h1', 'Cape')
h1.add_room(Room('Master Bedroom', 14, 21))
h1.add_room(Room('Living Room', 18, 20))
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))

h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))

h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))
houses = [h1, h2, h3]

print("Is h1 bigger than h2?", h1 > h2) # prints True
print("Is h2 smaller than h3?", h2 < h3) # prints True
print("Is h2 greater than or equal to h1?", h2 >= h1) # prints False
print("Which one is biggest?", max(houses)) # prints 'h3: 1101 square foot Split'
print("Which is smallest?", min(houses)) # prints 'h2: 846 square foot Ranch'


Is h1 bigger than h2? True
Is h2 smaller than h3? True
Is h2 greater than or equal to h1? False
Which one is biggest? h3: 1101 square foot Split
Which is smallest? h2: 846 square foot Ranch


### 8.25 Cached Instance

有些实例，如果创建时参数一致，可以不用重复创建，返回已经创建过的那个实例就行了。

这样做可以避免浪费内存。 

见过，比如  logger.getLogger('xxx')，如果名字一致，全局都是一样的。

In [120]:
class Spam:
    def __init__(self, name):
        self.name = name

# Caching support
import weakref
_spam_cache = weakref.WeakValueDictionary()

def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name] = s
    else:
        s = _spam_cache[name]
    return s

if __name__ == '__main__':
    a = get_spam('foo')
    b = get_spam('bar')
    print('a is b:', a is b)
    c = get_spam('foo')
    print('a is c:', a is c)
    print(a)
    print(b)
    print(c)

a is b: False
a is c: True


In [121]:
class Spam:
    _spam_cache = weakref.WeakValueDictionary()
    def __new__(cls, name):
        if name in cls._spam_cache:
            return cls._spam_cache[name]
        else:
            self = super().__new__(cls)
            cls._spam_cache[name] = self
            return self

    def __init__(self, name):
        print('Initializing Spam')
        self.name = name

if __name__ == '__main__':
    print("This should print 'Initializing Spam' twice")
    s = Spam('Dave')
    t = Spam('Dave')
    print(s is t)


This should print 'Initializing Spam' twice
Initializing Spam
Initializing Spam
True
